# MA934 Numerical Methods - Workbook 3

If you haven't already done so, install the DualNumbers Julia package. It is a good idea to update all your packages first. The commands are

>Pkg.update()

>Pkg.add("DualNumbers")

but you only need to run them once. 

In [2]:
# Pkg.update()
#Pkg.add("DualNumbers")
using Plots
using DualNumbers
include("MODULES/MyUtils_wb3.jl")

f_prime_ana (generic function with 1 method)

## Question 1: Numerical differentiation

**1))** Derive a finite difference formula for the derivative of a function, $f$ at a point $x$ using the 3-point stencil $(x, x+h, x+2h)$ and state the order of the approximation error in terms of $h$.

**2)** Write a formula for the derivative, $f^\prime(x)$, of the function

$$f(x) = \sin(\exp(x)) $$

and evaluate it at $x=1$.

**3)** Use your finite difference formula to approximate the value of $f^\prime(1)$ for values of $h$ decreasing from $2^{-1}$ to $2^{-30}$ in powers of $2$. Plot the error as a function of $h$ and verify the theoretically predicted scaling of the error with $h$. What is the best relative error you can achieve?

**4)** Read the examples at https://github.com/JuliaDiff/DualNumbers.jl. Define a dual number $x = 1+\epsilon$ and use it to evaluate $f^\prime(1)$. Verify that the answer is accurate to within machine precision.

1) 

We evaluate the function f(x) at the points of the stencil and taylor expand the function around these points:

\begin{align}
f(x)&=f(x)\\
f(x+h)&=f(x)+f'(x)h+f''(x)\frac{h^2}{2}+\mathcal{O}(h^3)\\
f(x+2h)&=f(x)+2f'(x)h+2f''(x){h^2}+\mathcal{O}(h^3)
\end{align}

We define F as a linear combination of the stencil: 

 $$F= A f(x) + B f(x+h) + C f(x+2h)$$


We want to use $F$ to approximate $f'(x)$, so we have to solve the following system of linear equations: 

\begin{align}
A+B+C&=0\\
B+2C&=1\\
\frac{B}{2}+2C&=0
\end{align}

Solving this gives $ A=-3/2 , B=2 , C=-1/2$

Finally, $F=f'(x)h+\mathcal{O}(h^3)$. Rearranging this suggests that 

$$ f'(x) = \frac{-3/2f(x)+2f(x+h)-1/2f(x+2h)}{h}+\mathcal{O}(h^2)$$

Therefore the order of the approximation error is $h^2$.


2) 
We want to evaluate the derivative of $f(x)=sin(exp(x))$ at $x=1$. 

$$ f'(x)=cos(exp(x))exp(x)$$

For $x=1$ we therefore get 

$$ f'(1)=cos(e)e$$

3)

We use our formula for $f'(x)$ which we found in part 1 and evaluate it at $x=1$ for $h$ from $2^{-1}$ to $2^{-30}$ in powers of 2. We plot the error as a function of $h$. 

In [3]:
function f(x)
   y=sin(exp(x))
    return y
end

function f_prime_num(h)
    x=1
    y=(-(3/2)*f(1)+2*f(x+h)-(1/2)*f(x+2*h))/h
    return y
end

f_prime_num (generic function with 1 method)

In [ ]:
der=cos(exp(1))*exp(1)
h=zeros(30)
z=zeros(30)
for i=1:30
    h[i]=2.0^(-i)
    z[i]=abs(f_prime_num(h[i])-der)
end

plot(h,z,linewidth=2, yscale=:log2, xscale=:log2,xlabel="h", label="Error",title="Error as a function of h [log/log plot]")


Initially we find that the error is decreasing with h, as expected, but as h gets too small (less than ~$2^{-17}$) the floating point arithmetics cause a significant rounding error. In the decreasing region, we see that the plot is approximately linear (on a log/log scale). Therefore we expect the error to go as a power law (error ~$h^k$) where $k$ is the gradient of the straight line. To find this gradient we only consider h from $2^{-1}$ to $2^{-17}$ and use the function "lingreg" to fit a line (linear regression). We find that gradient of the line is 2.13 , which roughly matches our expectations from part 1 (i.e. error $\mathcal{O}(h^2)$). 

The best relative error (minimum error) we can achieve is the minimum of the error function displayed above which equals $2^{-33.5}$.

In [ ]:
println(log2(findmin(z)[1]))

In [ ]:
h_cut=h[1:17]
z_cut=z[1:17]
h_cut_log=log2.(h_cut)
z_cut_log=log2.(z_cut)
a,b=linreg(h_cut_log,z_cut_log)
println("the gradient of the line is")
println(b)
plot(h_cut_log,z_cut_log,linewidth=2,label="numerical approximation",xlabel="log2(h)",ylabel="log2(error)",title="How does the error scale with h?")
plot!(h_cut_log,b*h_cut_log+a,linewidth=2,label="best fit line")

## Question 2: Finding roots

**1)** Referring to the function, $f(x)$, defined above, find the roots of the equation

$$ f(x) = 0$$

in the interval $0<x<2$.

**2)** Implement the bracketing and bisection method to find one of the roots numerically. Measure the error at each iteration of the algorithm and demonstrate that the error decreases exponentially as a function of the number of iterations. To how many digits of precision can you approximate the root?

**3)** Perform the same measurements for the Newton Raphson method and show that the error decreases faster than exponentially as a function of the number of iterations.

1)
We want to find the roots of the function $$f(x)=sin(exp(x))$$ 

For f(x) to be 0, $exp(x)$ has to be equal to $k\pi$ where $k \in \mathcal{Z}$. Solving $exp(x)=k\pi$ we get $x=log(k\pi)$. As we're only interested in solutions for $x \in [0,2]$, we solve $log(y)=0$ and $log(y)=2$ which gives us $k\pi \in [1, exp(2)\sim 7.4]$. Therefore our roots are $log(\pi)$ and $log(2\pi)$.

2) 
We want to find numerically find the root $log(\pi)$ of the function f(x) using the Bracketing and Bisection method. 

In [ ]:
steps=1:1:45
a=1.0
b=1.2
roots=zeros(length(steps))
error=zeros(length(steps))
for i in steps
    x=(a+b)/2
    if f(a)*f(x)>0 
        a=x
        b=b
    else
        a=a
        b=x
    end
    roots[i]=x
    error[i]=abs(log(pi)-x)
end

error_log=log10.(error)
a,b=linreg(steps,error_log)
println("Best fit line gradient =")
println(b)
plot(steps,error_log,linewidth=2,xlabel="iteration",ylabel="log10(error)",title="Error of the Bracketing and Bisection Method",label="log10(error)")
plot!(steps,b*steps+a,linewidth=2,label="best fit line")

As $log(\pi)$ is approximately $1.145$ we choose our inital interval to be $[a,b]=[1.0,1.2]$. We iterate the algorithm 45 times. The number of iterations is set to 45 as a bigger number of iterations (e.g. 50) turned out not be sensible due to the floating point arithmetics. We take the log10 of the calculated error and plot this against the iterations number. The plot is approximately linear (see bestfit line, gradient = -0.3) which implies that the error decreases exponantially with the iteration number. 

In [ ]:
println(roots[45])
log(pi)

Our best estimate of the root is 1.1447298858493982. The first 12 decimal places equal the ones of the analytical root $log(\pi)$.

3)

We now want to find the roots of the function using the Newton Raphson method. To do that, we use the analytics
derivative that we found in Q1 part 2: 

$$f'(x)=\cos(\exp(x))\exp(x)$$.

In [ ]:
function f_prime_ana(x)
    y=cos(exp(x))*exp(x)
    return y
end

steps=1:1:10
x=1.0
roots=zeros(length(steps))
error=zeros(length(steps))
for i in steps
    d=-f(x)/f_prime_ana(x)
    x=x+d
    roots[i]=x
    error[i]=abs(log(pi)-x)
end
println("The error values for the first 10 steps are")
println(error)
 error_log=log10.(error[1:4])
plot(steps[1:4],error_log,linewidth=2,ylabel="log10(error)",xlabel="iteration",title="Error of the Newton Raphson Method",label="log10(error)")
scatter!(steps[1:4],error_log,markersize=6,label="data points")

We want to find the root at $log(\pi)$ so we choose an inital value of x=1.0. The Newton Raphon algorithm finds the root extremely fast. Printing the error value for the first ten steps (see above), we see that the error is "equal" to zero after step 4.  We therefore only plot the first four datapoints. Plotting log10(error) again the iterations we see that the curve decays faster than a linear function implying a superexponential error decay. 

## Question 3: Finding minima

**1)** The function $f(x)$ above has a single minimum in the interval $0<x<2$. Find its location analytically.

**2)** Implement the Golden section search to find the location of this minimum numerically. Plot the error as a function of the number of iterations. To how many digits of precision can you approximate the location of the minimum?

**3)** To understand your empirical findings, use Taylor's Theorem to show that near a minimum, $x_*$, of f(x),

$$f(x) \approx f(x_*)\left( 1+ \frac{f^{\prime\prime}(x_*)}{2\,f(x_*)}\,(x-x_*)^2\right). $$
Show that in order for a computer to distinguish between $f(x)$ and $f(x_*)$ we must have

$$ \left| x-x_*\right| > \sqrt{\epsilon_m}\,\sqrt{\left|\frac{2\,f(x_*)}{f^{\prime\prime}(x_*)}\right|}$$

thus limiting the precision with which the location of a minimum can be determined.